In [1]:
import os

files_needed = [
    {"thinkplot.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkplot.py"},
    {"thinkstats2.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkstats2.py"},
]
current_folder = os.getcwd()
for f in files_needed:
    for file_name, url in f.items():
        if not os.path.exists(file_name):
            print(f"Downloading {file_name}")
            os.system(f"curl {url} -o {current_folder}/{file_name}")

In [2]:
import numpy as np
import random
import thinkstats2
import thinkplot
from scipy import stats as ss

##Seaborn for fancy plots.
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

<h1>Quiz 3</h1>

Please fill in the bodies of the functions as specified. Please read the instructions closely and ask for clarification if needed. A few notes/tips:
<ul>
<li>Like all the functions we use, the function is a self contained thing. It takes in values as paramaters when called, and produces a return value. All of the inputs that may change should be in that function call, imagine your function being cut/pasted into some other file - it should not depend on anything outside of libraries that it may need.
<li>Test your function with more than one function call, with different inputs. See an example in comments below the first question.
<li>If something doesn't work, print or look at the varaibles window. The #1 skill that'll allow you to write usable code is the ability to find and fix errors. Printing a value out line by line so you can see how it changes, and looking for the step where something goes wrong is A-OK and pretty normal. It is boring.
<li>Unless otherwise specified, you can use outside library functions to calculate things.
</ul>

<h1>Test Data</h1>

You may notice there's no data specified or attached. You'll need to generate some test data if you want to test your functions.

The easiest way to generate test data is to use some of the random functions to generate data that looks like what you need. Numpy random and scipy disributions .rvs functions are good places to look, we've also generated random data many times in the past.

There is no specific requirement on what your data needs to be, it just needs to be good enough to test your function. If you pay attention to what exactly you're calculating and the criteria given, you should be able to create some suitable data for different tests. As an example, for the Hyp Test question, you need two sets of normal data. You can generate some in many ways, one is through scipy:
<ul>
<li>ss.norm.rvs(loc=0, scale=1, size=1, random_state=None)
</ul>
<p>
Since you're checking if there's a significant difference between the two groups, you'd likely want multiple sets of data - two that are very close, so they will not show a difference, and two that are not close, so they will show a difference. Think about what you are checking, then just make some data that will allow you to test that.

This should not be extremely difficult to code nor should it be super time consuming, the commands are pretty simple and generating random varaibles is pretty similar for any distribution. There is some though involved in saying "what data do I need to check this?" That's something that is pretty important in general, if we are creating something we need to make sure that it works in general, not just one example. Critically, there are not specific sets of data you need - almost anything will work. It is only there to let your functions run and see if they are correct. You don't need to aim for "the perfect test data" or anything like that, just make some data in a list, if it needs to be of a certain distribution, use that dist to get it; if the distribution doesn't matter, just make something.

<h1>Ski on Chi - 10pts</h1>

You operate a ski hill, and over the years you've seen the distribution of skiers vs snowboarders vs snow skaters etc... change a bit. This is your first full open season since the pandemic hit. When you closed in early 2020, the distribution of your customer base was:
<ul>
<li>Skiers - 40%
<li>Snowboarders - 20%
<li>Snow Skaters - 5%
<li>Non-Active (i.e. sit in the lodger) - 15%
<li>Lesson takers - 20%
</ul>

You are seeing a different pattern now, but you are not sure if that is due to a change in what your customers want or due to just random chance. You want to be able to analytically tell if what you observe each week is a real change from that baseline above, or nothing to worry about.

In this function you'll take in:
<ul>
<li>Two list of values for the observed number of customers in each group, in the order indicated above. E.g. [35,25,10,10,20].
<li>An alpha value (the cutoff criteria for a p-values)
</ul>
<br><br>
You'll return 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha.
<li>The name of the category that MOST EXCEEDS the expectation.
<li>The name of the cetegory that is MOST EXCEEDED BY the expectation.
</ul>

In [7]:
def skiCustomersChange(observedCustys, alpha=.05):

    # We take the data from the problem question
    baseline_pct = np.array([0.40, 0.20, 0.05, 0.15, 0.20])
    categories = np.array(
        ["Skiers", "Snowboarders", "Snow Skaters", "Non-Active", "Lesson takers"])

    observed = np.array(observedCustys)
    total_customers = observed.sum()

    # This is the expected counts
    expected = baseline_pct * total_customers

    # Chi-square goodness-of-fit test
    chi2_stat, p_value = ss.chisquare(f_obs=observed, f_exp=expected)

    isSignificantDiff = p_value < alpha

    # Differences from expectation for each category
    diff_counts = observed - expected

    # Category most above / most below expectation
    higherThanExp = categories[diff_counts.argmax()]
    lowerThanExp  = categories[diff_counts.argmin()]

    return isSignificantDiff, higherThanExp, lowerThanExp

diff, highCategory, lowCategory = skiCustomersChange([35,25,10,10,20], .05)
print(diff, highCategory, lowCategory)

# Once we run this we will be able to see T/F and which categories exceeds the
# expectation and exceeded by the expectation

False Snowboarders Skiers


In [4]:
#Example function calls
# diff, highCategory, lowCategory = skiCustomersChange([35,25,10,10,20], .05)
# diff, highCategory, lowCategory = skiCustomersChange([15,40,15,10,20], .1)
# diff, highCategory, lowCategory = skiCustomersChange([40,10,10,10,30], .01)

<h2>Hypothesis Testing - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB. The data will be normally distributed.
<li>An alpha value (the cutoff criteria for a p-values)
<li>A power value (the likelihood of not getting a false negative)
<li>An effect size value.
</ul>
<br><br>
You'll produce a tuple of 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha in a t-test.
<li>A true/false assessment for if a hypothesis test has enough power to be reliable, measured by if the power you calculate is greater than the supplied power.
<li>A true false assessment for if the data appears to show a significant difference in means, measured by if the Cohen effect size is greater than the supplied effect size.
</ul>

<b>Please report your responses in the format indicated in the template. As well, please report all true/false values as 1/0. 1 is True, 0 is false. To verify if all the criteria are true, someone calling this function should be able to multiply the 3 values in the tuple together and get a result of 1 if they are all true, and 0 otherwise</b>

In [36]:
def strengthOfEffect(dataA, dataB, alpha=.05, power=.8, effectSize=.5):

    # Convert lists to numpy arrays
    a = np.array(dataA)
    b = np.array(dataB)

    # This is the t-test for significant difference in means
    t_stat, p_value = ss.ttest_ind(a, b, equal_var=False)
    passedPtest = 1 if p_value < alpha else 0

    # Now we made a simple power calculation for our code
    pooled_std = np.sqrt(((a.var(ddof=1) + b.var(ddof=1)) / 2))
    cohen_d = abs(a.mean() - b.mean()) / pooled_std

    analysis = TTestIndPower()
    calc_power = analysis.solve_power(effect_size=cohen_d, nobs1=len(a), alpha=alpha,
    ratio=len(b) / len(a), alternative='two-sided')

    passedPower = 1 if calc_power > power else 0

    # Effect size comparison
    passedEffectSize = 1 if cohen_d > effectSize else 0

    results = (passedPtest, passedPower, passedEffectSize)
    return results

    dataA = [10, 12, 11, 13, 9]
    dataB = [20, 19, 21, 18, 22]

print(strengthOfEffect(dataA, dataB, alpha=.05, power=.8, effectSize=.5))

NameError: name 'dataA' is not defined

In [ ]:
#Example function calls
# results = strengthOfEffect(oneListOfValues, anotherListOfValues, .05, .9, .7)
# results = strengthOfEffect(secondListOfValues, anotherListOfValues, .03, .7, .4)
# results = strengthOfEffect(oneListOfValues, moreListOfValues, .05, .8, .7)

<h2>Safe Test - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB.
</ul>
<br><br>
You'll produce a p-value for a two sided hypothesis test:
<ul>
<li>If the data is not normally distributed, use a Mann-Whitney Test.
<li>If the data appears to be normally distributed, and the variance differs substantially, use a Welch's t-test.
<li>If none of those conditions are true, use a 'normal' (Student's) t-test.
<li>Note: The execution of all of these tests are very similar from your persepective. They are all in the scipy documentation - Google for exact details, and the code closely mirrors the examples we did.
<li>Note 2: If you ever need to use a cutoff for a p-value in the middle of your calculations, please choose something reasonable. There are common defaults for whatever you may need. These defaults are likely shown in the documentation or any examples you may look up.
</ul>

<b>In any case, the value returned is one number (not in a list, tuple, etc...) that is the pValue performed for that test.

In [25]:
def flexHypTest(dataA, dataB):
  a = np.array(dataA)
  b = np.array(dataB)

    # Check normality Shapiro-Wilk
    statA, pA = ss.shapiro(a)
    statB, pB = ss.shapiro(b)
    alpha_norm = 0.05

    if pA < alpha_norm or pB < alpha_norm:
        # Not normal using Mann-Whitney U test
        stat, pvalue = ss.mannwhitneyu(a, b, alternative='two-sided')
    else:
        # Normal check variance equality Levene's test
        stat_lev, p_lev = ss.levene(a, b)
        alpha_var = 0.05

        if p_lev < alpha_var:
            # Variances differ using Welch's t-test
            stat, pvalue = ss.ttest_ind(a, b, equal_var=False)
        else:
            # Variances similar using Student's t-test
            stat, pvalue = ss.ttest_ind(a, b, equal_var=True)

    return pvalue

# Lastly we want the output
print(flexHypTest([1, 2, 3, 4, 5], [2, 3, 4, 5, 6]))


0.34659350708733416


<h1>Grade Distribution - 10pts</h1>

Grade distributions for final letter grades at a school are generally skewed towards the higher end of the scale. We can model it with a function below.

Percentage grades on individual assignments are often skewnormally distributed. (Note: this is more for curved schools than somewhere like NAIT with hard cutoffs. When I was in school CompSci profs would aim for a 50%-60% raw average to get a normal-ish distribution of marks.)

You are seeking to generate a grading system, in two steps:
<ul>
<li>Use the supplied Weibull distribution in the simpleGenerateLetterGradeBuckets function to generate the distribution of letter grades - A,B,C,D,F. We are a simple school and we only have letters, no plus or minus.
<li>
<li>Use the function simpleGenerateLetterGradeBuckets to tell you HOW MANY slots there are for each grade. This is done for you in the provided function, you just need to call it and get the results. Please pay attention to the n value for number.
<li>Take the supplied raw percentage grades and fit them into those buckets. I.E. if there are 17 slots for an A grade, the 17 highest percentage marks should get an A; if there are then 52 for B, then the next 52 highest get a B, etc...
<li><b>You are going to return a list of tuples - the original percentage grade, and the letter grade. E.g. [(72,B), (84,A), etc...]</b>
</ul>

<br><br>
In this function you'll take in:
<ul>
<li>A list of raw percentage grades, from 0 to 100. E.g. [100,98,24,53,45, etc...]
</ul>

You'll produce:
<ul>
<li>A list of tuples. Each tuple is the original percentage grade, and the letter grade. .
</ul>

<br>
Note: You'll have to run the function cell down at the bottom first.
<br><br>
<b>Bonus: The provided function for grade buckets probably isn't the best overall, if you can rewrite it to be better, up to 3 bonus marks. Think about the random factor...</b>

In [34]:
def assignLetterGrades(rawPercentageGrades):
    # First we get bucket sizes for A,B,C,D,F
    gradeBuckets = simpleGenerateLetterGradeBuckets()

    # 2. Sort grades from highest to lowest
    sortedGrades = sorted(rawPercentageGrades, reverse=True)

    # 3.Then Assign grades according to bucket sizes
    letterAssignments = {}
    index = 0

    for letter in ['A', 'B', 'C', 'D', 'F']:
        letterAssignments[letter] = sortedGrades[index : index + gradeBuckets[letter]]
        index += gradeBuckets[letter]

    # 4. Create the output tuples (original grade, letter grade)
    result = []

    for grade in rawPercentageGrades:   # keep original order
        assignedLetter = None
        for letter in letterAssignments:
            if grade in letterAssignments[letter]:
                assignedLetter = letter
                letterAssignments[letter].remove(grade)  # avoid duplicates
                break
        result.append((grade, assignedLetter))

    return result

print(assignLetterGrades([100, 98, 92, 90]))



[(100, 'A'), (98, 'A'), (92, 'A'), (90, 'A')]


In [ ]:
#Example for 423 students
simpleGenerateLetterGradeBuckets(423)

{'A': 65, 'B': 173, 'C': 121, 'D': 46, 'F': 18}

In [37]:
def simpleGenerateLetterGradeBuckets(n=100):
    #Define distribution params
    c = 1.5
    loc = 3
    scale = 1.5

    # Define cuts for 'data' values (7.2 - X), then convert to X-space for CDF calculation
    # A: 7.2 - X > 3.7  => X < 3.5
    # B: 7.2 - X > 2.9  => X < 4.3
    # C: 7.2 - X > 1.9  => X < 5.3
    # D: 7.2 - X > 0.9  => X < 6.3
    # F: 7.2 - X <= 0.9 => X >= 6.3
    cut_points_x = [3.5, 4.3, 5.3, 6.3]

    # Calculate CDF at these points to get theoretical probabilities for each grade
    cdf_values = [ss.weibull_min.cdf(x, c, loc, scale) for x in cut_points_x]

    # Calculate theoretical probabilities for each grade category
    # P(X < 3.5)
    prob_A = cdf_values[0]
    # P(3.5 <= X < 4.3)
    prob_B = cdf_values[1] - cdf_values[0]
    # P(4.3 <= X < 5.3)
    prob_C = cdf_values[2] - cdf_values[1]
    # P(5.3 <= X < 6.3)
    prob_D = cdf_values[3] - cdf_values[2]
    # P(X >= 6.3)
    prob_F = 1.0 - cdf_values[3]

    probs = [prob_A, prob_B, prob_C, prob_D, prob_F]
    grade_letters = ["A", "B", "C", "D", "F"]
    buckets = {letter: 0 for letter in grade_letters}

    remaining_n = n

    # Calculate initial bucket sizes by rounding and ensure sum is n
    for i, letter in enumerate(grade_letters):
        count = int(round(probs[i] * n))
        buckets[letter] = count
        remaining_n -= count

    # Distribute any remaining students due to rounding errors
    # This ensures the sum of buckets is exactly n
    if remaining_n != 0:
        # Find the grade with the largest proportion to adjust
        # This is a simple heuristic to distribute rounding error
        max_prob_idx = np.argmax(probs)
        buckets[grade_letters[max_prob_idx]] += remaining_n

    return buckets